In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [5]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()
model

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [22]:
outputs= []
def layer1_hook(module, input_, output):
    "自行填入"
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [23]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [24]:
outputs

[tensor([[[[1.5045, 0.0000, 0.5335,  ..., 0.0000, 0.2868, 0.8787],
           [0.3841, 0.6391, 0.6382,  ..., 0.4410, 0.1977, 0.4160],
           [0.9209, 0.9750, 0.6026,  ..., 0.5733, 0.7386, 0.1956],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.9682, 0.0223, 0.7576],
           [0.1209, 0.0000, 0.7828,  ..., 0.4687, 0.7731, 0.4050],
           [0.0000, 0.2658, 0.9185,  ..., 0.2679, 0.3963, 0.1274]],
 
          [[0.9673, 0.1357, 1.4003,  ..., 1.5116, 0.4540, 0.8260],
           [0.7637, 0.3204, 0.8507,  ..., 1.2690, 0.5721, 1.0376],
           [0.0000, 0.9668, 0.0000,  ..., 0.2329, 1.2097, 0.9206],
           ...,
           [0.3804, 0.7822, 0.1726,  ..., 0.2353, 0.8564, 0.6173],
           [2.0799, 1.2260, 0.7961,  ..., 1.4619, 0.0000, 1.2806],
           [0.6042, 0.8828, 0.1630,  ..., 1.0213, 0.6147, 0.4145]],
 
          [[0.6715, 0.4312, 0.4970,  ..., 0.3363, 0.6882, 0.8563],
           [1.0611, 0.0000, 0.7401,  ..., 0.8867, 1.3504, 0.3688],
           [0.8714, 1.32

In [25]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 6, 14, 14])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [28]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            "自行填入"
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [29]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


